In [1]:
import os

from language_table.environments import blocks
from language_table.environments import language_table
from language_table.environments.rewards import block2block

from matplotlib import pyplot as plt
import mediapy
import numpy as np
import tensorflow_datasets as tfds
from icecream import ic

np.set_printoptions(precision=3)

pybullet build time: Nov 28 2023 23:48:36
2024-06-04 04:37:21.422877: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 04:37:22.703012: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# tf dataset handling
import tensorflow as tf

dataset = tf.data.Dataset.range(10)

for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


2024-06-04 04:37:28.611217: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-04 04:37:28.615647: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-04 04:37:28.770499: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required l

In [3]:
DATASET_PATH = "/data/language_table_sim"
builder = tfds.builder_from_directory(DATASET_PATH)
episode_ds = builder.as_dataset(split="train")
print(episode_ds.element_spec)

{'episode_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'steps': DatasetSpec({'action': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'observation': {'effector_target_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'effector_translation': TensorSpec(shape=(2,), dtype=tf.float32, name=None), 'instruction': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'rgb': TensorSpec(shape=(360, 640, 3), dtype=tf.uint8, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}


In [4]:
def decode_inst(inst):
    """Utility to decode encoded language instruction"""
    return bytes(inst[np.where(inst != 0)].tolist()).decode("utf-8")

In [5]:
def show_episode(episode):
    """Utility to visualize an episode"""
    print("Episode id:", episode["episode_id"].numpy())
    frames = []
    actions = []
    for step in episode["steps"].as_numpy_iterator():
        frames.append(step["observation"]["rgb"])
        actions.append(step["action"])

    print("step numbers: ", len(frames))
    mediapy.show_video(
        frames, title=decode_inst(step["observation"]["instruction"]), fps=5
    )
    # draw the action sequence
    actions = np.array(actions)
    actions.min(axis=0)
    print("action sequence:")
    print(actions)
    return actions.min(axis=0), actions.max(axis=0)

In [6]:
episodes_iter = iter(episode_ds.take(50))

# sample_episodes = tf.data.Dataset.sample_from_datasets(episode_ds)
# episodes_iter = iter(sample_episodes)
action_mins = []
action_maxes = []

for episode in episodes_iter:
    action_min, action_max = show_episode(episode)
    action_mins.append(action_min)
    action_maxes.append(action_max)

action_mins = np.array(action_mins)
action_maxes = np.array(action_maxes)

print("global action min:", action_mins.min(axis=0))
print("global action max:", action_maxes.max(axis=0))

Episode id: b'EiIKGHdvcmtlcl8wODlfZXBfOF8wNl8wNl8yMhAvGDggBDAAKiBmNTI1ODUyODdhMTU3NmI4NWRmYmMyOTliNDk4MTFmYw=='
step numbers:  10


2024-06-04 04:37:44.301217: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


surround the red pentagon with the others This browser does not support the video tag.


action sequence:
[[-3.057e-04  1.751e-02]
 [-7.094e-03  9.966e-03]
 [-1.165e-02  2.447e-03]
 [-1.404e-02 -7.375e-05]
 [-7.023e-03 -2.571e-03]
 [-4.685e-03 -2.567e-03]
 [ 8.861e-05 -5.076e-03]
 [-2.658e-04  1.523e-02]
 [-3.987e-04  2.284e-02]
 [ 0.000e+00  0.000e+00]]
Episode id: b'EiMKGXdvcmtlcl8xMDZfZXBfMTBfMDZfMDZfMjIQORhrIA0wASogZjUyNTg1Mjg3YTE1NzZiODVkZmJjMjk5YjQ5ODExZmM='
step numbers:  51


2024-06-04 04:37:44.512544: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

action sequence:
[[-1.354e-02 -8.802e-03]
 [-1.693e-02 -5.417e-03]
 [-1.286e-02  1.255e-09]
 [-4.062e-03 -1.354e-03]
 [ 0.000e+00 -6.771e-03]
 [-5.551e-17 -2.708e-03]
 [ 5.551e-17 -3.385e-03]
 [ 1.219e-02 -7.448e-03]
 [ 6.771e-04  5.791e-10]
 [-2.167e-02 -1.563e-08]
 [-1.828e-02  5.417e-03]
 [-9.479e-03  1.016e-02]
 [-3.385e-03  4.062e-03]
 [ 2.031e-03  3.385e-03]
 [ 1.354e-03  1.354e-03]
 [ 5.120e-11 -6.771e-04]
 [ 0.000e+00  0.000e+00]]
Episode id: b'EiMKGHdvcmtlcl8wMzlfZXBfMl8wNl8xN18yMhAAGCIgADDpByogZDRkNTFjYzE0YzI0OWY0ODA2MTRlMzUwM2FmZThiZDU='
step numbers:  35


2024-06-04 04:37:52.532802: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


"put the blue blocks in the bottom right corner, the red blocks in the bottom center, the green blocks in the center right, and the yellow blocks in the bottom left corner. This browser does not support the video tag."


action sequence:
[[ 2.694e-03 -4.873e-05]
 [ 2.834e-02  9.830e-03]
 [ 2.944e-02  5.763e-03]
 [ 2.974e-02  3.963e-03]
 [ 2.998e-02  1.010e-03]
 [ 2.971e-02 -4.153e-03]
 [ 1.412e-02 -3.915e-03]
 [ 4.315e-03 -3.197e-03]
 [ 2.673e-03 -2.525e-03]
 [ 1.241e-02 -3.054e-04]
 [ 1.069e-02  2.759e-03]
 [ 8.019e-03  7.001e-03]
 [ 5.854e-03  6.404e-03]
 [ 2.155e-03  4.109e-03]
 [ 2.249e-03  8.873e-03]
 [-1.395e-03  7.240e-03]
 [-7.869e-03  2.084e-02]
 [-6.379e-03  2.096e-02]
 [-3.878e-03  1.667e-02]
 [-1.915e-03  9.888e-03]
 [ 6.349e-05  3.031e-03]
 [-1.453e-03  6.831e-03]
 [ 6.583e-04  6.013e-03]
 [ 7.662e-03  1.189e-02]
 [ 9.208e-03  1.458e-02]
 [ 2.792e-03  9.259e-03]
 [ 2.354e-03  1.348e-02]
 [ 1.206e-03  8.222e-03]
 [ 1.743e-03  9.327e-03]
 [ 1.139e-03  5.212e-03]
 [ 5.628e-04  2.305e-03]
 [ 1.249e-05  5.965e-04]
 [-6.657e-03  2.864e-03]
 [-8.287e-03  3.144e-03]
 [ 0.000e+00  0.000e+00]]
global action min: [-0.03 -0.03]
global action max: [0.03 0.03]


2024-06-04 04:37:52.714980: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
